In [2]:
# use svg graphics, display inline
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import glob
import re
import copy
import sys

# basic scientific computing imports
import pandas as pd
import numpy as np

# ai
from tqdm.notebook import tnrange, tqdm
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

np.random.seed(42)

# Don't usually do this but OpenAI an langchain throw lots of warnings
import warnings
warnings.filterwarnings("ignore")

print(sys.version)

3.9.16 | packaged by conda-forge | (main, Feb  1 2023, 21:38:11) 
[Clang 14.0.6 ]


# Objective
Create a program that designs a creative family photoshoot for every US holiday

# Steps
- Generate a list of us holidays and dates
- Generate a photoshoot for each holiday
- Generate a midjourney prompt for each shoot
- Generate a shopping list for each one

In [3]:
def convert_string_to_tuples(holiday_string):
    return [tuple(line.split(', ')) for line in holiday_string.split('\n')]

In [4]:
generate_holidays_prompt = '''
Generate a list of all widely celebrated US holidays. Include unofficial but fun holidays like Halloween and Father's day.
Be sure to include Lunar New Year (use an approximate date if necessary)
Include a few major South Korean holidays as well.

Example:
01/01, New Year's Day
10/31, Halloween

List of Holidays:
'''

llm = OpenAI(model_name='gpt-4', temperature=0.4)
holidays_string = llm(generate_holidays_prompt)
holidays_data = convert_string_to_tuples(holidays_string)

holidays_data

[('01/01', "New Year's Day"),
 ('01/18', 'Martin Luther King Jr. Day'),
 ('02/12', 'Lunar New Year (approximate date)'),
 ('02/14', "Valentine's Day"),
 ('03/17', "St. Patrick's Day"),
 ('04/04', 'Easter (approximate date)'),
 ('05/09', "Mother's Day"),
 ('05/31', 'Memorial Day'),
 ('06/19', 'Juneteenth'),
 ('06/20', "Father's Day"),
 ('07/04', 'Independence Day'),
 ('09/06', 'Labor Day'),
 ('10/11', 'Columbus Day'),
 ('10/31', 'Halloween'),
 ('11/11', 'Veterans Day'),
 ('11/25', 'Thanksgiving Day'),
 ('12/24', 'Christmas Eve'),
 ('12/25', 'Christmas Day'),
 ('',),
 ('South Korean Holidays:',),
 ('01/01', "New Year's Day"),
 ('03/01', 'Independence Movement Day'),
 ('05/05', "Children's Day"),
 ('06/06', 'Memorial Day'),
 ('08/15', 'Liberation Day'),
 ('09/20', 'Chuseok (approximate date)'),
 ('10/03', 'National Foundation Day'),
 ('10/09', 'Hangul Proclamation Day')]

### Clean up the list

In [6]:
holidays_data = [x for x in holidays_data if '/' in x[0] and len(x) == 2]
holidays_data.sort(key=lambda x: x[0])

### Generate the data

In [8]:
generate_photoshoot_prompt = '''
### Task
Describe a creative, tasteful portrait photo that incorporates a given US holiday.
The subject is an adorable 7 month old baby girl.
The photoshoot concept should be creative and simple.

### Holiday
{holiday}

### Photoshoot concept
'''


photoshoot_concepts = []
for _, holiday in tqdm(holidays_data):
    # I think this prevents context from accumulating, which is desirable here
    llm = OpenAI(model_name='gpt-4', temperature=0.4)
    shoot_concept = llm(generate_photoshoot_prompt.format(holiday=holiday))
    photoshoot_concepts.append(shoot_concept)
    

# photoshoot_concepts[0]

  0%|          | 0/26 [00:00<?, ?it/s]

In [12]:
generate_mj_prompt_prompt = '''
### Task
Convert a description of a photoshoot into a more descriptive photoshoot prompt that is similar to the example prompt. Ensure that it includes the following:
- A mention of the subject, a 7 month old baby girl
- A description of the scene and the pose
- A description of the camera and it's settings

Ensure that the prompt is no longer than 5 sentences!

### Example Output Prompt
Set within a cozy, naturally lit room, adorned with gentle touches of fantastical elements like soft, ethereal fairy lights, and gauzy pastel-hued fabrics. Our model, the 7-month old baby, lies comfortably on a plush, cloud-shaped pillow, adorned with a tiny crown, capturing the essence of a fantasy prince/princess. For lighting, large windows allow the soft glow of early morning light to pour in, supplemented with a softbox for balanced illumination. Canon EOS R5, 50mm f/1.2 lens, sharp details, smooth bokeh effect, 1/250 shutter speed, f/2.8 aperture, ISO 200.

### Input description
{description}

### Output Prompt
'''



prompts = []
for shoot in tqdm(photoshoot_concepts):
    llm = OpenAI(model_name='gpt-3.5-turbo', temperature=0.4)
    prompt = llm(generate_mj_prompt_prompt.format(description=shoot))
    prompts.append(prompt)
    

# prompts[0]

  0%|          | 0/26 [00:00<?, ?it/s]

In [17]:
generate_shopping_list_prompt = '''
### Task
Generate a list of things a photographer would need to buy to put on the following photoshoot.

### Photoshoot description
{description}

### Shopping list
'''

shopping_lists = []
for shoot in tqdm(photoshoot_concepts):
    llm = OpenAI(model_name='gpt-3.5-turbo', temperature=0.4)
    shopping_list = llm(generate_shopping_list_prompt.format(description=shoot))
    shopping_lists.append(shopping_list)
    

# shopping_lists[0]

  0%|          | 0/26 [00:00<?, ?it/s]

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', TimeoutError(60, 'Operation timed out')).


In [21]:
df = (
    pd.DataFrame({
        'date': [x[0] for x in holidays_data],
        'holiday': [x[1] for x in holidays_data],
        'photoshoot_description': photoshoot_concepts,
        'midjourney_prompt': prompts,
        'shopping_list': shopping_lists,
    })
    .assign(
        title=lambda d: d.photoshoot_description.str.split('"', expand=True)[1],
        photoshoot_description=lambda d: d.photoshoot_description.str.split('\n\n', n=1, expand=True)[1]
    )
    [['date', 'holiday', 'title', 'photoshoot_description', 'midjourney_prompt', 'shopping_list']]
)

# df.sort_values('date').to_clipboard(sep='\t', index=False)
# df.to_excel()
df

,date,holiday,title,photoshoot_description,midjourney_prompt,shopping_list
0,01/01,New Year's Day,Baby's First New Year's Sparkle,Concept: For this New Year's Day themed photos...,"Our star of the portrait, a 7-month-old baby g...","1. White or light gray backdrop\n2. Cute, whit..."
1,01/01,New Year's Day,A New Year's Baby Dream,Concept: For this New Year's Day themed photos...,Our 7-month-old baby girl will be the star of ...,"1. Soft, pastel-colored canvas or printed fabr..."
2,01/18,Martin Luther King Jr. Day,Dreaming of a Brighter Future,Concept: This portrait photo will be a tribute...,With a light blue sky background and fluffy wh...,"1. Light blue backdrop fabric\n2. Soft, fluffy..."
3,02/12,Lunar New Year (approximate date),Baby Blossoms for Lunar New Year,"Concept: For this photoshoot, we will incorpor...","Incorporating the Lunar New Year theme, we wil...",1. Silk cherry blossoms\n2. Small red lanterns...
4,02/14,Valentine's Day,Cupid's Littlest Love,Concept: For this Valentine's Day themed photo...,Transform your 7-month-old baby girl into a li...,"1. Soft, pastel pink backdrop\n2. Tiny white h..."
5,03/01,Independence Movement Day,Little Lady Liberty,Concept: For this Independence Movement Day th...,Our little Lady Liberty photoshoot will captur...,"1. Soft, green onesie\n2. Small, lightweight, ..."
6,03/17,St. Patrick's Day,Luck of the Irish Baby,Concept: For this St. Patrick's Day-themed pho...,Surrounded by elements of Irish folklore and c...,"1. Soft, pastel green backdrop fabric or hand-..."
7,04/04,Easter (approximate date),Easter Blossoms,"Concept: For this Easter-themed photoshoot, th...",Surrounded by the vibrant colors and symbols o...,1. Camera\n2. Lens (preferably a portrait lens...
8,05/05,Children's Day,Blooming Joy on Children's Day,Concept: For this Children's Day-themed photos...,"Surrounded by a mini flower garden, our 7-mont...",1. Canvas backdrop in soft shades of lavender ...
9,05/09,Mother's Day,Mommy's Little Flower,Concept: For this Mother's Day themed photosho...,"""Little Flower Blossom""\n\nTransform the 7-mon...",1. Light pastel-colored fabric for backdrop\n2...
